In [5]:
from transformers import BloomForCausalLM
from transformers import BloomTokenizerFast
import torch

checkpoint = "ckip-joint/bloom-1b1-zh"
model = BloomForCausalLM.from_pretrained(checkpoint)
tokenizer = BloomTokenizerFast.from_pretrained(checkpoint)

print('模型總參數量 :', model.num_parameters()) # 查看模型總參數量
model

模型總參數量 : 1065314304


BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 1536)
    (word_embeddings_layernorm): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=1536, out_features=4608, bias=True)
          (dense): Linear(in_features=1536, out_features=1536, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=1536, out_features=6144, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=6144, out_features=1536, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
  )
  (

In [2]:
# 出現 cuda 環境編譯錯誤，參考這裡 : https://www.baifachuan.com/posts/543c16ad.html
# 出現警告 : https://github.com/oobabooga/text-generation-webui/issues/1164 (執行pip install -i https://test.pypi.org/simple/ bitsandbytes-cuda113)

# 使用 LoRA 網路
from peft import LoraConfig, get_peft_model, TaskType, get_peft_model_state_dict

# Define LoRA Config
lora_config = LoraConfig(
 peft_type="LORA",
 task_type=TaskType.CAUSAL_LM,  # task_type, token classification (TaskType.CAUSAL_LM)
 inference_mode=False,
 r=16,                           # r, the dimension of the low-rank matrices
 lora_alpha=64,                 # lora_alpha, scaling factor for the weight matrices
 lora_dropout=0.05,              # lora_dropout, dropout probability of the LoRA layers
#  bias="lora_only"               # bias, set to only lora layers to train
)

# prepare int-8 model for training
# model = prepare_model_for_int8_training(model)

# add LoRA adaptor
model = get_peft_model(model, lora_config)
model.config.use_cache = False

model.print_trainable_parameters()

trainable params: 2,359,296 || all params: 1,067,673,600 || trainable%: 0.22097539922313336


In [3]:
# 觀察可訓練參數
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"Parameter: {name} is not frozen")
    else:
        print(f"Parameter: {name} is frozen")

Parameter: base_model.model.transformer.word_embeddings.weight is frozen
Parameter: base_model.model.transformer.word_embeddings_layernorm.weight is frozen
Parameter: base_model.model.transformer.word_embeddings_layernorm.bias is frozen
Parameter: base_model.model.transformer.h.0.input_layernorm.weight is frozen
Parameter: base_model.model.transformer.h.0.input_layernorm.bias is frozen
Parameter: base_model.model.transformer.h.0.self_attention.query_key_value.base_layer.weight is frozen
Parameter: base_model.model.transformer.h.0.self_attention.query_key_value.base_layer.bias is frozen
Parameter: base_model.model.transformer.h.0.self_attention.query_key_value.lora_A.default.weight is not frozen
Parameter: base_model.model.transformer.h.0.self_attention.query_key_value.lora_B.default.weight is not frozen
Parameter: base_model.model.transformer.h.0.self_attention.dense.weight is frozen
Parameter: base_model.model.transformer.h.0.self_attention.dense.bias is frozen
Parameter: base_model.m

In [4]:
from transformers import BloomForCausalLM
from transformers import BloomTokenizerFast
import torch

from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

model = None
hf_peft_repo = "/HDD2/p76124388/1015_lora_bloom_1b1_zh/final"
peft_config = PeftConfig.from_pretrained(hf_peft_repo)
model = AutoModelForCausalLM.from_pretrained(peft_config.base_model_name_or_path, return_dict=True)
tokenizer = AutoTokenizer.from_pretrained(peft_config.base_model_name_or_path)

# Load the finetuned Lora PEFT model
model = PeftModel.from_pretrained(model, hf_peft_repo)

model

ValueError: Can't find 'adapter_config.json' at '/HDD2/p76124388/1015_lora_bloom_1b1_zh/final'